<a href="https://colab.research.google.com/github/huangale88/hydro-download/blob/main/hydro_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [149]:
# @title Install Libraries {"display-mode":"form"}
%%capture
!pip install folium
!pip install geopy
!pip install pyproj
!pip install geopandas
# !pip install panel
# !pip install jupyter_bokeh
!pip install clipboard
!pip install pyperclip
!pip install owslib
!pip install cartopy

In [150]:
# @title Import Libraries {"display-mode":"form"}
import folium
from geopy.geocoders import Nominatim
from pyproj import CRS, Transformer
from IPython.display import display, HTML
from folium.plugins import Geocoder
from folium.plugins import MiniMap
from folium.plugins import MeasureControl
from folium.plugins import Draw
from folium import Figure
import requests
import geopandas as gpd
from datetime import datetime, date
import json
import math
from textwrap import fill
import cartopy.crs as ccrs
from cartopy.io.img_tiles import OSM
from matplotlib import pyplot as plt, dates as mdates
from osgeo import ogr, osr
from owslib.ogcapi.features import Features
import numpy as np
import pandas as pd
from tabulate import tabulate
import plotly.express as px
import plotly.graph_objects as go

In [151]:
# @title Show Map
# Create the map and add features
fig = Figure(width=1000, height=600)
m = folium.Map(location=[49.1386, -123.0139], zoom_start=10)
folium.TileLayer("Esri.WorldImagery").add_to(m)
folium.TileLayer("OpenStreetMap").add_to(m)
folium.TileLayer("OpenTopoMap").add_to(m)
m.add_child(folium.ClickForMarker("<b>Lat:</b> ${lat}<br /><b>Lon:</b> ${lng}"))
m.add_child(folium.ClickForLatLng(format_str='"[" + lat + "," + lng + "]"', alert=False))
Geocoder().add_to(m)

# Add WMS layer to the same map instance
folium.WmsTileLayer(
    url="https://geo.weather.gc.ca/geomet-climate/?service=WMS&version=1.3.0&request=GetCapabilities",
    name="Climate",
    fmt="image/png",
    layers="CLIMATE.STATIONS",
    attr=u"stations",
    transparent=True,
    overlay=True,
    control=True,
).add_to(m)

folium.WmsTileLayer(
    url="https://geo.weather.gc.ca/geomet-climate/?service=WMS&version=1.3.0&request=GetCapabilities",
    name="Hydrometric",
    fmt="image/png",
    layers="HYDROMETRIC.STATIONS",
    attr=u"stations",
    transparent=True,
    overlay=True,
    control=True,
).add_to(m)

# Add a collapsible minimap
MiniMap(toggle_display=True).add_to(m)

# Add a measuring tool
m.add_child(MeasureControl())

'''
# Add a draw function
Draw(export=True).add_to(m)
file_path = 'your_file.geojson'
m.save(file_path)
'''

# Add LayerControl after all layers have been added
folium.LayerControl().add_to(m)

fig.add_child(m)

In [152]:
Coordinates = "[49.253913,-122.663040]" #@param {type:"string"}
Coordinates = Coordinates.strip("[]")
lat, long = [float(coord) for coord in Coordinates.split(",")]
Distance = "20" #@param {type:"string"}
buffer = float(Distance)

# Get the EPSG projection
def get_epsg_code(lat, lon):
    # Create a CRS object for WGS 84
    crs_wgs84 = CRS("EPSG:4326")

    # Determine the UTM zone
    utm_zone = int((lon + 180) // 6) + 1
    hemisphere = 'north' if lat >= 0 else 'south'

    # Create a CRS object for the UTM zone in NAD83
    crs_utm = CRS.from_proj4(f"+proj=utm +zone={utm_zone} +datum=NAD83 +units=m +no_defs")

    # Get the EPSG code
    epsg_code = crs_utm.to_epsg()

    return epsg_code

projection = get_epsg_code(lat, long)

In [153]:
# @title Call API
# Parameters formatting for the OGC API - Features request

# Bounding box a little bigger than buffer size

# The buffer needs to be transformed in degrees to get
# the coordinates of the corners of the bounding box:
# Latitude: 1 km ≈ 0.009°
# Longitude (at the 49th parallel): 1 km ≈ 0.014°
bbox = [
    long - buffer * 0.02,
    lat - buffer * 0.01,
    long + buffer * 0.02,
    lat + buffer * 0.01,
]

# Retrieval of stations data
oafeat = Features("https://api.weather.gc.ca/")
station_data = oafeat.collection_items(
    "climate-stations", bbox=bbox
)

# Verification of the retrieved data
if "features" in station_data:
    station_data = json.dumps(station_data, indent=4)
else:
    raise ValueError(
        "No stations were found. Please verify the coordinates."
    )

In [154]:
# @title Search for Stations
# List of stations located inside the buffer zone

# Accessing the hydrometric stations layer
driver = ogr.GetDriverByName("GeoJSON")
data_source = driver.Open(station_data, 0)
layer = data_source.GetLayer()

# Identification of the input spatial reference system (SRS)
SRS_input = layer.GetSpatialRef()
SR = osr.SpatialReference(str(SRS_input))
epsg = SR.GetAuthorityCode(None)
SRS_input.ImportFromEPSG(int(epsg))

# Definition of the SRS used to project data
SRS_projected = osr.SpatialReference()
SRS_projected.ImportFromEPSG(projection)

# Transformation from input SRS to the prefered projection
transform = osr.CoordinateTransformation(SRS_input, SRS_projected)

# Creation of a buffer to select stations
point = ogr.Geometry(ogr.wkbPoint)
point.AddPoint(long, lat)
point.Transform(transform)
point_buffer = point.Buffer(buffer * 1000)  # The value must be in meters

# Create an empty list to store station data as dictionaries
station_data_list = []

for feature in layer:
    geom = feature.GetGeometryRef().Clone()
    geom.Transform(transform)
    if geom.Intersects(point_buffer):
        # Get all feature field names
        field_names = [feature.GetFieldDefnRef(i).GetName() for i in range(feature.GetFieldCount())]

        # Get feature field values
        field_values = [feature.GetField(name) for name in field_names]

        # Create a dictionary representing the station data
        station_data = dict(zip(field_names, field_values))

        # Append the dictionary to the list
        station_data_list.append(station_data)

# Create a Pandas DataFrame from the list of dictionaries
stations_df = pd.DataFrame(station_data_list)

# Correct latitude and longitude values
stations_df['LATITUDE'] = stations_df['LATITUDE'] / 10000000
stations_df['LONGITUDE'] = stations_df['LONGITUDE'] / 10000000

# Raising an error if no station were found
if not station_data_list:
    raise ValueError(
        f"There are no stations within {buffer} km"
        + " of the chosen coordinates. Please verify the coordinates."
    )

In [155]:
# @title Data Retrieval
# Retrieval of data for each station

# Dictionary that will contain a data frame for each station
climate_data = {}

# List of stations with no data
stations_without_data = []

# Data retrieval and creation of the data frames
for station in stations_df['STN_ID']:

    # Set the start and end daily data dates to be automatically picked from the df
    start_string = stations_df.loc[stations_df['STN_ID'] == station, 'DLY_FIRST_DATE'].iloc[0]
    start_date  = datetime.strptime(start_string, '%Y/%m/%d %H:%M:%S').date()
    end_string = stations_df.loc[stations_df['STN_ID'] == station, 'DLY_LAST_DATE'].iloc[0]
    end_date  = datetime.strptime(end_string, '%Y/%m/%d %H:%M:%S').date()

    # Retrieval of climate data
    station_data = oafeat.collection_items(
        "climate-daily",
        bbox=bbox,
        datetime=f"{start_date}/{end_date}",
        STN_ID=station,
    )

    # Check if data was retrieved
    if station_data["features"]:

        # Creation of a dictionary in a format compatible with Pandas
        historical_data_format = [
            {
                "LATITUDE": el["geometry"]["coordinates"],
                "LONGITUDE": el["geometry"]["coordinates"],
                **el["properties"],
            }
            for el in station_data["features"]
        ]

        # Creation of the data frame
        historical_data_df = pd.DataFrame(
            historical_data_format,
            columns=[
                "CLIMATE_IDENTIFIER",
                "STATION_NAME",
                "LOCAL_DATE",
                "TOTAL_PRECIPITATION",
                "TOTAL_RAIN",
                "TOTAL_SNOW",
                "MEAN_TEMPERATURE",
            ],
        )
        historical_data_df = historical_data_df.fillna(value=np.nan)

        # Adding the data frame to the climate data dictionary
        if not historical_data_df.isnull().all().any():
            # Removing any rows without climate data at the
            # end of the data frame
            '''
            while np.isnan(historical_data_df["TOTAL_PRECIPITATION"].iloc[-1]):
                historical_data_df = historical_data_df.drop(
                    historical_data_df.tail(1).index
                )
            '''
            # Creating an index with the date in a datetime format
            historical_data_df["LOCAL_DATE"] = pd.to_datetime(
                historical_data_df["LOCAL_DATE"]
            )
            historical_data_df.set_index(["LOCAL_DATE"], inplace=True, drop=True)
            historical_data_df.index = historical_data_df.index.date
            historical_data_df.sort_index(inplace=True)
            # Adding the data frame to the dictionary
            climate_data[station] = historical_data_df
        # If all the data is NaN, the station will be removed from the dataset
        else:
            stations_without_data.append(station)
            print(f"No valid data for station: {station}")

    # If there is no data for the chosen time period, the station
    # will be removed from the dataset
    else:
        stations_without_data.append(station)

# Removing stations without data from the station list
stations_df = stations_df[~stations_df['STN_ID'].isin(stations_without_data)]


<ipython-input-155-9dd8c33f1786>:53: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



No valid data for station: 674


<ipython-input-155-9dd8c33f1786>:53: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

<ipython-input-155-9dd8c33f1786>:53: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



No valid data for station: 683
No valid data for station: 684


<ipython-input-155-9dd8c33f1786>:53: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

<ipython-input-155-9dd8c33f1786>:53: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



...

<ipython-input-155-9dd8c33f1786>:53: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

<ipython-input-155-9dd8c33f1786>:53: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



No valid data for station: 6829
No valid data for station: 6830


<ipython-input-155-9dd8c33f1786>:53: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

<ipython-input-155-9dd8c33f1786>:53: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



No valid data for station: 27007
No valid data for station: 27198
No valid data for station: 53278
No valid data for station: 55460
No valid data for station: 55462


<ipython-input-155-9dd8c33f1786>:53: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

<ipython-input-155-9dd8c33f1786>:53: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

<ipython-input-155-9dd8c33f1786>:53: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [172]:
# @title Timeseries Plots
# Combine all dataframes into one
df_combined = pd.concat(climate_data.values())

# Create a list of columns available for plotting
columns = df_combined.columns

# Initial plot with 'TOTAL_PRECIPITATION'
fig = px.line(df_combined, x=df_combined.index, y='TOTAL_PRECIPITATION', color='STATION_NAME', markers=True)

# Update the layout to have all traces initially hidden
fig.update_traces(visible='legendonly')

# Create buttons for the dropdown menu
dropdown_buttons = [
    {'label': column,
     'method': 'update',
     'args': [{'y': [df_combined[column]]}, {'yaxis': {'title': column}}]}
    for column in columns[2:]
]

# Update layout with dropdown menu
fig.update_layout(
    updatemenus=[
        {
            'buttons': dropdown_buttons,
            'direction': 'down',
            'showactive': True
        }
    ]
)

fig.show()